In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from os import path
import os
import glob
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

from IPython.display import clear_output
from IPython.display import display

from collections import defaultdict
from datetime import datetime

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)

import warnings
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#Source address data
data = pd.read_csv('/Users/michaelwilliams/Documents/MGH T32/Research/AB/Analysis/AB_Rural/Analytic Dataset/Address_Data/Post_Geocode/AB_Addresses_PostGeocode.csv')
print(data.shape)
display(data.head(2))

#RUCA
#Rural Urban Communiting Area Data
ruca = pd.read_csv('/Users/michaelwilliams/Documents/NEU PhD/Research/SDOHE/Data/Census/Rural Urban Commuting/Tract/RUCA-codes-2020-tract.csv', 
                   converters={'TractFIPS23': lambda x: str(x), 'TractFIPS20': lambda x: str(x)},
                   encoding='cp1252')
print(ruca.shape)
display(ruca.head(2))

#2023 Census tract shapefile
tract_2023_shp = gpd.read_file('/Users/michaelwilliams/Documents/NEU PhD/Research/SDOHE/Data/Census/Geography/Shapefiles/tracts/2023/nhgis0006_shapefile_tl2023_us_tract_2023.zip')
tract_2023_shp = tract_2023_shp.to_crs('EPSG:4326')
print(tract_2023_shp.crs)

(281, 68)


,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_record_id,USER_addr_baseline,USER_addr_current,USER_id_AB
0,1,M,95.87,A,"Putney Lndg, Putney, Vermont, 05346","Putney Lndg, Putney, VT, 05346, USA",Putney Lndg,StreetName,,,"Putney Lndg, Putney, Vermont, 05346",,,20,,,,,,,,,Putney,Lndg,,,,,,,,,,Putney Lndg,,,,,Putney,,Windham County,Vermont,VT,,,5346.0,,USA,United States,ENG,47520.881,-72.519971,42.962967,-72.519971,42.962967,-72.520971,-72.518971,42.961967,42.963967,,,,,"PO Box 208 Putney, VT 05346",1,"PO Box 208 Putney, VT 05346","PO Box 208 Putney, VT 05346",1_AB
1,2,M,100.00,A,"314 Cottage Ct, Cloverdale, California, 95425","314 Cottage Ct, Cloverdale, CA, 95425, USA",314 Cottage Ct,PointAddress,,,"314 Cottage Ct, Cloverdale, California, 95425",,,20,,314,,,,,,,Cottage,Ct,,,,,,,,,,314 Cottage Ct,,,,,Cloverdale,,Sonoma County,California,CA,,,95425.0,4436,USA,United States,ENG,4235596.049,-123.019465,38.783223,-123.019613,38.783429,-123.020613,-123.018613,38.782429,38.784429,,,Residential,Single Family,"314 Cottage Court Cloverdale, CA 95425",4,"314 Cottage Court Cloverdale, CA 95425","314 Cottage Court Cloverdale, CA 95425",4_AB


(85528, 27)


,TractFIPS23,CountyFIPS23,CountyCode23,CountyName23,TractFIPS20,TractCode20,TractName20,CountyFIPS20,CountyCode20,CountyName20,StateFIPS20,StateName20,UrbanAreaCode20,UrbanAreaName20,UrbanCore,UrbanCoreType,PrimaryRUCA,PrimaryRUCADescription,PrimaryDestinationCode,PrimaryDestinationName,SecondaryRUCA,SecondaryRUCADescription,SecondaryDestinationCode,SecondaryDestinationName,Population,LandArea,PopDensity
0,01001020100,1001.0,1.0,Autauga County,01001020100,20100,Census Tract 201,1001,1,Autauga County,1,Alabama,58600.0,"Montgomery, AL",1,Metro core,1,Metropolitan core,58600.0,"Montgomery, AL",1.0,"Metropolitan core, no addtional code",1.101006e+09,Census Tract 59.04; Montgomery County; Alabama,1775,3.8,467.9
1,01001020200,1001.0,1.0,Autauga County,01001020200,20200,Census Tract 202,1001,1,Autauga County,1,Alabama,58600.0,"Montgomery, AL",1,Metro core,1,Metropolitan core,58600.0,"Montgomery, AL",1.0,"Metropolitan core, no addtional code",1.101006e+09,Census Tract 59.04; Montgomery County; Alabama,2055,1.3,1602.7


EPSG:4326


In [ ]:
## Quick data checks on post-geocoded address data
# record 93, 222 gave PO box only and the geocoder couldn't match those to an address
#data[data['Score'] < 85] #

#data[~data["Match_type"].isin(["A", "T"])]

#data[data['Status'] != 'M']

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_record_id,USER_addr_baseline,USER_addr_current,USER_id_AB
90,91,U,0.00,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,PO Box 351 Oak City North Caroline 27857,93,PO Box 351 Oak City North Caroline 27857,PO Box 351 Oak City North Caroline 27857,93_AB
135,136,T,99.17,A,"739 Daniel Shays Highway, Athol, Massachusetts, 01331","739 Daniel Shays Highway, Athol, MA, 01331, USA",739 Daniel Shays Highway,PointAddress,NaN,NaN,"739 Daniel Shays Highway, Athol, Massachusetts, 01331",NaN,NaN,20,NaN,739,NaN,NaN,NaN,NaN,NaN,NaN,Daniel Shays,Highway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,739 Daniel Shays Highway,NaN,NaN,NaN,NaN,Athol,NaN,Franklin County,Massachusetts,MA,NaN,NaN,1331.0,9325,USA,United States,ENG,0.0,-72.276569,42.574747,-72.276569,42.574747,-72.276569,-72.276569,42.574747,42.574747,B30,NaN,NaN,NaN,739 Daniel Shays Highway B30 Athol MA 01331,150,739 Daniel Shays Highway B30 Athol MA 01331,739 Daniel Shays Highway B30 Athol MA 01331,150_AB
203,204,U,0.00,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,PO 86 Milaca MN 56353,222,PO 86 Milaca MN 56353,PO 86 Milaca MN 56353,222_AB
234,235,T,100.00,A,"95 Kestrel Dr, Falling Waters, West Virginia, 25419","95 Kestrel Dr, Falling Waters, WV, 25419, USA",95 Kestrel Dr,PointAddress,NaN,NaN,"95 Kestrel Dr, Falling Waters, West Virginia, 25419",NaN,NaN,20,NaN,95,NaN,NaN,NaN,NaN,NaN,NaN,Kestrel,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95 Kestrel Dr,NaN,NaN,NaN,NaN,Falling Waters,NaN,Berkeley County,West Virginia,WV,NaN,NaN,25419.0,4327,USA,United States,ENG,0.0,-77.896500,39.567279,-77.897354,39.567198,-77.898354,-77.896354,39.566198,39.568198,NaN,NaN,Residential,Single Family,"95 Kestrel Dr, Falling Waters WV 25419",256,"95 Kestrel Dr, Falling Waters WV 25419","95 Kestrel Dr, Falling Waters WV 25419",256_AB


In [19]:
#Check to see if tract FIPS changed from 2020 to 2023 anywehere
#Some did, 883 tracts
ruca[ruca['TractFIPS23'] != ruca['TractFIPS20']].shape

ruca['UrbanCoreType'].value_counts()

UrbanCoreType
Metro core         59247
Rural              18367
Micro core          5525
Small town core     2021
Water                368
Name: count, dtype: int64

In [8]:
#Spatial Join
#construct geoDataframe
geodata = gpd.GeoDataFrame(data,
    geometry=gpd.points_from_xy(data.X, data.Y),
    crs=4326)

data_sj = gpd.sjoin(geodata, tract_2023_shp, how='left', predicate='within')

data_sj.head()
data_sj["GEOID"].astype(str).str.len().value_counts()

GEOID
11    279
3       2
Name: count, dtype: int64

In [ ]:
# Two PO box records didn't merge, is expected
data_sj[data_sj['GEOID'].isnull()]

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_record_id,USER_addr_baseline,USER_addr_current,USER_id_AB,geometry,index_right,GISJOIN,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area,ORIG_FID
90,91,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,PO Box 351 Oak City North Caroline 27857,93,PO Box 351 Oak City North Caroline 27857,PO Box 351 Oak City North Caroline 27857,93_AB,POINT (0 0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,204,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,PO 86 Milaca MN 56353,222,PO 86 Milaca MN 56353,PO 86 Milaca MN 56353,222_AB,POINT (0 0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

#Merge with RUCA info
data_final = data_sj.merge(ruca,how='left', left_on='GEOID', right_on='TractFIPS23', indicator='ruca_merge')

#Again, two po boxes not merging, this is expected
display(data_final[data_final['ruca_merge'] != 'both'])
display(data_final.head(2))

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_record_id,USER_addr_baseline,USER_addr_current,USER_id_AB,geometry,index_right,GISJOIN,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area,ORIG_FID,TractFIPS23,CountyFIPS23,CountyCode23,CountyName23,TractFIPS20,TractCode20,TractName20,CountyFIPS20,CountyCode20,CountyName20,StateFIPS20,StateName20,UrbanAreaCode20,UrbanAreaName20,UrbanCore,UrbanCoreType,PrimaryRUCA,PrimaryRUCADescription,PrimaryDestinationCode,PrimaryDestinationName,SecondaryRUCA,SecondaryRUCADescription,SecondaryDestinationCode,SecondaryDestinationName,Population,LandArea,PopDensity,ruca_merge
90,91,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,PO Box 351 Oak City North Caroline 27857,93,PO Box 351 Oak City North Caroline 27857,PO Box 351 Oak City North Caroline 27857,93_AB,POINT (0 0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
203,204,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,PO 86 Milaca MN 56353,222,PO 86 Milaca MN 56353,PO 86 Milaca MN 56353,222_AB,POINT (0 0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_record_id,USER_addr_baseline,USER_addr_current,USER_id_AB,geometry,index_right,GISJOIN,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area,ORIG_FID,TractFIPS23,CountyFIPS23,CountyCode23,CountyName23,TractFIPS20,TractCode20,TractName20,CountyFIPS20,CountyCode20,CountyName20,StateFIPS20,StateName20,UrbanAreaCode20,UrbanAreaName20,UrbanCore,UrbanCoreType,PrimaryRUCA,PrimaryRUCADescription,PrimaryDestinationCode,PrimaryDestinationName,SecondaryRUCA,SecondaryRUCADescription,SecondaryDestinationCode,SecondaryDestinationName,Population,LandArea,PopDensity,ruca_merge
0,1,M,95.87,A,"Putney Lndg, Putney, Vermont, 05346","Putney Lndg, Putney, VT, 05346, USA",Putney Lndg,StreetName,,,"Putney Lndg, Putney, Vermont, 05346",,,20,,,,,,,,,Putney,Lndg,,,,,,,,,,Putney Lndg,,,,,Putney,,Windham County,Vermont,VT,,,5346.0,,USA,United States,ENG,47520.881,-72.519971,42.962967,-72.519971,42.962967,-72.520971,-72.518971,42.961967,42.963967,,,,,"PO Box 208 Putney, VT 05346",1,"PO Box 208 Putney, VT 05346","PO Box 208 Putney, VT 05346",1_AB,POINT (-72.51997 42.96297),77908.0,G5000250967700,50,025,967700,50025967700,1400000US50025967700,9677,Census Tract 9677,G5020,S,69044547.0,180757.0,+42.9951916,-072.5298301,36950.705086,6.922530e+07,83245.0,50025967700,50025.0,25.0,Windham County,50025967700,967700.0,Census Tract 9677,50025.0,25.0,Windham County,50.0,Vermont,NaN,NaN,0.0,Rural,10.0,Rural area,5.002597e+10,Census Tract 9677; Windham County; Vermont,10.0,"Rural area, no additional code",9703.0,"Brattleboro, VT",2617.0,26.7,98.0,both
1,2,M,100.00,A,"314 Cottage Ct, Cloverdale, California, 95425","314 Cottage Ct, Cloverdale, CA, 95425, USA",314 Cottage Ct,PointAddress,,,"314 Cottage Ct, Cloverdale, California, 95425",,,20,,314,,,,,,,Cottage,Ct,,,,,,,,,,314 Cottage Ct,,,,,Cloverdale,,Sonoma County,California,CA,,,95425.0,4436,USA,United States,ENG,4235596.049,-123.019465,38.783223,-123.019613,38.783429,-123.020613,-123.018613,38.782429,38.784429,,,Residential,Single Family,"314 Cottage Court Cloverdale, CA 95425",4,"314 Cottage Court Cloverdale, CA 95425","314 Cottage Court Cloverdale, CA 95425",4_AB,POINT (-123.01947 38.78322),12772.0,G0600970154202,06,097,154202,06097154202,1400000US06097154202,1542.02,Census Tract 1542.02,G5020,S,175450308.0,14292823.0,+38.7861401,-123.1049627,78241.919696,1.897431e+08,7687.0,06097154202,6097.0,97.0,Sonoma County,06097154202,154202.0,Census Tract 1542.02,6097.0,97.0,Sonoma County,6.0,California,18181.0,"Cloverdale, CA",1.0,Small town core,2.0,Metropolitan high commuting,7.949800e+04,"Santa Rosa, CA",2.0,"Metropolitan high commuting, no additional code",18181.0,"Cloverdale, CA",6306.0,67.5,93.5,both


In [21]:
data_final.to_csv('AB_Rural_Analytic.csv', index=False)


In [23]:
#data_final